# Data Engineer Challenge – EDA

Antes de meterme a resolver las preguntas, quiero entender bien con que estoy trabajando.  
El dataset viene de tweets sobre el Farmers Protest en India

lo primero es ver que estructura tiene cada tweet, que campos me sirven para cada pregunta,  
y si hay datos faltantes que puedan complicar las cosas.


In [1]:
import os
import orjson
import pandas as pd
import emoji
from datetime import datetime
from collections import Counter
from typing import List, Tuple

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)

FILE_PATH = os.path.join('..', 'tweets.json', 'farmers-protest-tweets-2021-2-4.json')

size_mb = os.path.getsize(FILE_PATH) / 1024 / 1024
print(f'Archivo: {os.path.basename(FILE_PATH)}')
print(f'Tamaño: {size_mb:.1f} MB')

Archivo: farmers-protest-tweets-2021-2-4.json
Tamaño: 388.8 MB


## que tiene adentro cada tweet?

voy a leer la primera línea del archivo para ver la estructura completa.  
Cada línea es un JSON independiente

In [2]:
with open(FILE_PATH, 'rb') as f:
    sample = orjson.loads(f.readline())

# Veamos qué campos tiene cada tweet
for key, value in sample.items():
    tipo = type(value).__name__
    preview = str(value)[:60] if value is not None else 'None'
    print(f'{key:<22} ({tipo:<8}) → {preview}')

url                    (str     ) → https://twitter.com/ArjunSinghPanam/status/13645062492917841
date                   (str     ) → 2021-02-24T09:23:35+00:00
content                (str     ) → The world progresses while the Indian police and Govt are st
renderedContent        (str     ) → The world progresses while the Indian police and Govt are st
id                     (int     ) → 1364506249291784198
user                   (dict    ) → {'username': 'ArjunSinghPanam', 'displayname': 'Arjun Singh 
outlinks               (list    ) → ['https://twitter.com/ravisinghka/status/1364150844757860352
tcooutlinks            (list    ) → ['https://t.co/es3kn0IQAF']
replyCount             (int     ) → 0
retweetCount           (int     ) → 0
likeCount              (int     ) → 0
quoteCount             (int     ) → 0
conversationId         (int     ) → 1364506249291784198
lang                   (str     ) → en
source                 (str     ) → <a href="http://twitter.com/download/iphone" rel="

Me interesan particularmente tres cosas:
El objeto user (para Q1 y saber quién tweetea)
El content (para Q2, extraer emojis)
La lista mentionedUsers (para Q3)

In [3]:
# extrae el usuario que hizo el tweet
print('usuario del tweet ')
for k, v in sample['user'].items():
    print(f'  {k}: {str(v)[:50]}')

print()

# extrae los usuarios mencionados
print('usuarios mencionados')
if sample.get('mentionedUsers'):
    for u in sample['mentionedUsers']:
        print(f'  @{u["username"]}')
else:
    print('  (ninguno en este tweet)')

print()
print('--- Contenido ---')
print(sample['content'])

usuario del tweet 
  username: ArjunSinghPanam
  displayname: Arjun Singh Panam
  id: 45091142
  description: Global Citizen, Actor, Director: Sky is the roof a
  rawDescription: Global Citizen, Actor, Director: Sky is the roof a
  descriptionUrls: []
  verified: False
  created: 2009-06-06T07:50:57+00:00
  followersCount: 603
  friendsCount: 311
  statusesCount: 17534
  favouritesCount: 4269
  listedCount: 23
  mediaCount: 1211
  location: 
  protected: False
  linkUrl: https://www.cosmosmovieofficial.com
  linkTcourl: https://t.co/3uaoV3gCt3
  profileImageUrl: https://pbs.twimg.com/profile_images/1215541746492
  profileBannerUrl: https://pbs.twimg.com/profile_banners/45091142/161
  url: https://twitter.com/ArjunSinghPanam

usuarios mencionados
  @narendramodi
  @DelhiPolice

--- Contenido ---
The world progresses while the Indian police and Govt are still trying to take India back to the horrific past through its tyranny. 

@narendramodi @DelhiPolice Shame on you. 

#ModiDontSellFarm

## estado general del dataset

Hago una pasada completa leyendo línea por línea (streaming) para tener un panorama  
sin cargar los ~389 MB en memoria de golpe.

In [4]:
%%time

total_tweets = 0
tweets_con_menciones = 0
tweets_con_emojis = 0
idiomas = Counter()
fecha_min, fecha_max = None, None

with open(FILE_PATH, 'rb') as f:
    for line in f:
        t = orjson.loads(line)
        total_tweets += 1
        
        if t.get('mentionedUsers'):
            tweets_con_menciones += 1
        if emoji.emoji_count(t.get('content', '')) > 0:
            tweets_con_emojis += 1
        
        d = t.get('date', '')
        if d:
            if not fecha_min or d < fecha_min: fecha_min = d
            if not fecha_max or d > fecha_max: fecha_max = d

print(f'total de tweets: {total_tweets:,}')
print(f'periodo: {fecha_min[:10]} ----> {fecha_max[:10]}')
print(f'con menciones: {tweets_con_menciones:,} ({tweets_con_menciones/total_tweets*100:.1f}%)')
print(f'con emojis: {tweets_con_emojis:,} ({tweets_con_emojis/total_tweets*100:.1f}%)')

total de tweets: 117,407
periodo: 2021-02-12 ----> 2021-02-24
con menciones: 38,034 (32.4%)
con emojis: 16,874 (14.4%)
CPU times: total: 7.94 s
Wall time: 7.91 s


hay 117,407 tweets en 13 días de febrero

## Pregunta 1: Top 10 fechas con más tweets

Para esto necesito:

la fecha ------> saco los primeros 10 caracteres del campo date ([:10])
el username ---> esta en tweet[user][username]

por cada fecha cuento tweets totales, y además llevo un Counter de usernames para sacar el top.

In [5]:
%%time

tweets_x_fecha = Counter()
users_x_fecha = {}

with open(FILE_PATH, 'rb') as f:
    for line in f:
        t = orjson.loads(line)
        fecha = t['date'][:10]
        user = t['user']['username']
        
        tweets_x_fecha[fecha] += 1
        if fecha not in users_x_fecha:
            users_x_fecha[fecha] = Counter()
        users_x_fecha[fecha][user] += 1

print(f'top 10 fechas (de {len(tweets_x_fecha)} unicas):\n')
for fecha, n in tweets_x_fecha.most_common(10):
    
    print(f'  {fecha}  ===>  {n:>6,} tweets ')

top 10 fechas (de 13 unicas):

  2021-02-12  ===>  12,347 tweets 
  2021-02-13  ===>  11,296 tweets 
  2021-02-17  ===>  11,087 tweets 
  2021-02-16  ===>  10,443 tweets 
  2021-02-14  ===>  10,249 tweets 
  2021-02-18  ===>   9,625 tweets 
  2021-02-15  ===>   9,197 tweets 
  2021-02-20  ===>   8,502 tweets 
  2021-02-23  ===>   8,417 tweets 
  2021-02-19  ===>   8,204 tweets 
CPU times: total: 859 ms
Wall time: 855 ms


el dia 12 de febrero fue el dia pico con aproximadamente 12 mil tweets.

aclaracion:  en caso de empate de tweets entre usuarios en un mismo dia, tomo el primero  
que devuelve el Counter (el que mas aparece; si hay empate exacto, es arbitrario).

---

## Pregunta 2: Top 10 emojis mas usados

Uso la librería emoji que detecta emojis Unicode
Con emoji.emoji_list() saco la lista de cada emoji que aparece en el texto.

In [6]:
%%time

conteo_emojis = Counter()

with open(FILE_PATH, 'rb') as f:
    for line in f:
        t = orjson.loads(line)
        for e in emoji.emoji_list(t.get('content', '')):
            conteo_emojis[e['emoji']] += 1

print(f'Emojis distintos encontrados: {len(conteo_emojis)}\n')
for e, n in conteo_emojis.most_common(10):
    print(f'  {e}  {n:>6,}  {emoji.demojize(e)}')

Emojis distintos encontrados: 870

  🙏   5,049  :folded_hands:
  😂   3,072  :face_with_tears_of_joy:
  🚜   2,972  :tractor:
  🌾   2,182  :sheaf_of_rice:
  🇮🇳   2,086  :India:
  🤣   1,668  :rolling_on_the_floor_laughing:
  ✊   1,651  :raised_fist:
  ❤️   1,382  :red_heart:
  🙏🏻   1,317  :folded_hands_light_skin_tone:
  💚   1,040  :green_heart:
CPU times: total: 7.77 s
Wall time: 7.77 s


### benchmark: deteccion de emojis

antes de elegir el enfoque final, compare tres metodos para detectar emojis:

- emoji.emoji_list()  ~ 7 segundos es exacta presicion

- regex (\p{Emoji})  ~ 0.5 segundos  9/10 coinciden (falla con skin tones)

- emoji.analyze()  ~ 8 segundos exacta  pero algo mas de tiempo

- el regex es 14x mas rapido pero no detecta correctamente emojis compuestos
(banderas, skin tones, ). ajustando el patron se llega a 9/10
coincidencias, pero no al 100%

**conclusion:** elijo emoji.emoji_list() para ambas variantes porque la precision
en la deteccion es mas importante que la velocidad. la diferencia entre _time
y _memory la hago en la gestion de memoria:


- q2_time precarga todo el contenido en lista 38.8 mb
- q2_memory streaming linea a linea  0.9 mb

97.6% de ahorro de memoria con resultados identicos.

el emoji de las manos rezando 🙏 es el mss usado , seguido de 😂 y 🚜 

aclaracion a tener en cuenta: cada aparicion cuenta por separado, por lo que si un tweet tiene 🙏🙏🙏, eso suma 3.

---

## Pregunta 3: Top 10 usuarios mas mencionados

aca uso el campo mentionedUsers que ya viene parseado en el JSON, es mas confiable que hacer regex sobre el texto

In [8]:
%%time

conteo_menciones = Counter()

with open(FILE_PATH, 'rb') as f:
    for line in f:
        t = orjson.loads(line)
        mencionados = t.get('mentionedUsers')
        if mencionados:
            for u in mencionados:
                if u.get('username'):
                    conteo_menciones[u['username']] += 1

print(f'Usuarios distintos mencionados: {len(conteo_menciones)}\n')
for user, n in conteo_menciones.most_common(10):
    print(f'  @{user:<22} {n:>5,}')

Usuarios distintos mencionados: 15239

  @narendramodi           2,265
  @Kisanektamorcha        1,840
  @RakeshTikaitBKU        1,644
  @PMOIndia               1,427
  @RahulGandhi            1,146
  @GretaThunberg          1,048
  @RaviSinghKA            1,019
  @rihanna                  986
  @UNHumanRights            962
  @meenaharris              926
CPU times: total: 953 ms
Wall time: 962 ms


@narendramodi es el mas mencionado con ~2,265 menciones

uso mentionedUsers en lugar de buscar con regex en el contenido, porque es mas preciso

---

chequeo de datos faltantes: antes de implementar las funciones, me aseguro de que los campos que necesito esten todos completos.

In [9]:
campos = {'date': 0, 'user': 0, 'user.username': 0, 'content': 0, 'mentionedUsers (null/vacío)': 0}
total = 0

with open(FILE_PATH, 'rb') as f:
    for line in f:
        t = orjson.loads(line)
        total += 1
        if not t.get('date'): campos['date'] += 1
        if not t.get('user'): campos['user'] += 1
        elif not t['user'].get('username'): campos['user.username'] += 1
        if not t.get('content'): campos['content'] += 1
        if not t.get('mentionedUsers'): campos['mentionedUsers (null/vacío)'] += 1

print(f'registros totales: {total:,}\n')
for campo, nulos in campos.items():
    print(f'  {campo}: {nulos:,} ({nulos/total*100:.1f}%)')

registros totales: 117,407

  date: 0 (0.0%)
  user: 0 (0.0%)
  user.username: 0 (0.0%)
  content: 0 (0.0%)
  mentionedUsers (null/vacío): 79,373 (67.6%)


Los campos criticos (date, user, username, content) estan completos al 100%.  
mentionedUsers es null/vacio en ~67.6%, pero eso es logico: no todos los tweets mencionan a alguien.

---

## resumen de enfoques

| pregunta | campo | _time | _memory | diferencia real |
|----------|-------|-------|---------|----------------|
| Q1 – top fechas | date, user.username | 1 pasada, guarda todos los users por fecha | 2 pasadas, solo users de top 10 | memoria: menos Counters |
| Q2 – top emojis | content | precarga contenido en lista (39 MB) | streaming linea a linea (0.9 MB) | memoria: 97.6% ahorro |
| Q3 – top menciones | mentionedUsers[].username | streaming + Counter | streaming + Counter | minima (dato liviano) |

## aclaraciones finales

1. extraigo la fecha como string ([:10]) sin parsear a datetime, es mas rapido

2. para emojis uso `emoji.emoji_list()` — probamos regex (14x mas rapido) pero falla con emojis compuestos. la precision importa mas

3. prefiero `mentionedUsers` sobre regex porque ya viene pre-parseado

4. en empates, tomo lo que el Counter devuelva primero (sin criterio de desempate adicional)

---

# ejecucion de funciones de solucion


In [10]:

# from q1_time import q1_time
# from q1_memory import q1_memory
# from q2_time import q2_time
# from q2_memory import q2_memory
# from q3_time import q3_time
# from q3_memory import q3_memory

In [11]:
# q1
# %timeit q1_time(FILE_PATH)
# %memit q1_time(FILE_PATH)
# %timeit q1_memory(FILE_PATH)
# %memit q1_memory(FILE_PATH)

In [12]:
# q2
# %timeit q2_time(FILE_PATH)
# %memit q2_time(FILE_PATH)
# %timeit q2_memory(FILE_PATH)
# %memit q2_memory(FILE_PATH)

In [13]:
# q3
# %timeit q3_time(FILE_PATH)
# %memit q3_time(FILE_PATH)
# %timeit q3_memory(FILE_PATH)
# %memit q3_memory(FILE_PATH)